In [ ]:
import datetime
from validate import *

In [1]:
from mrequests import MattermostRequests
from yamlvalidator import validate_response
from test_flows import MattermostTestFlows
from jsonfuzzer import JSONFuzzer

In [2]:
# Mattermost credentials should to be placed outside of this repo.
mr = MattermostRequests('../mattermost_credentials.json', 'mattermost-openapi-v4.yaml')
flows = MattermostTestFlows(mr)

In [3]:
ctf = flows.create_user_flow()
fuzzer = JSONFuzzer()

#### API testing

A motor feladata, hogy végrehajt egy teszt sort, és értékeli az eredményt. Addig folyik a tesztelés, amíg egy hibás válasz nem érkezik. (Hiba az is, ha nem érkezik válasz egy megadott időn belül, de itt figyelembe kell venni a hálózati késleltetést is.) A fuzzingot ezen a ponton úgy kéne megvalósítani, hogy az előbbi fuzzoló metódust használva a teszt alatt lévő API metódus össze lehetséges paraméterét végigtesztelni az összes lehetséges kombinációban. (Ez elvben nagyon sok, úgyhogy inkább az összes lehetséges teszt egy random részhalmazával érdemes próbálkozni.) A visszajövő válaszokat pedig minden esetben a séma alapján validálni kell (lásd 2. feladat).

In [4]:
def run_test(flow, n = 1):
    issue_list = []
    cache = {}

    for action in flow['setup'] if 'setup' in flow else []:
        input_data = action['input_data'] if 'input_data' in action else {}
        input_data.update(cache)
        target = action['target']
        post_process = action['post_process']
        cache.update(post_process(target(**input_data)))
    
    print(cache)

    for k in range(n):
        for i, action in enumerate(flow['test']):
            print('Test ' + str(i + 1) + '/' + str(len(flow['test'])))
            
            # prepare variables
            input_data = action['input_data'] if 'input_data' in action else {}
            input_data.update(cache)
            target = action['target']
            
            # fuzzing input
            method_params = mr.handler.get_matching_methods() 
            input_data = fuzzer().fuzz(method_params)
            print('Input: ' + str(input_data))
            
            # run target
            request, response = target(**input_data)
            _, result = response
            print('Output: ' + str(result))
            
            # validate result
            
            # prepare result for next 
            if 'post_process' in action:
                post_process = action['post_process']
                result_data = post_process(result) if post_process is not None else {'result': result}
                cache.update(result_data)
        
            # save
            issue_list.append({'Input':input_data, 'Output':result})
    
    print(cache)
    
    for action in flow['teardown'] if 'teardown' in flow else []:
        input_data = action['input_data'] if 'input_data' in action else {}
        input_data.update(cache)
        target = action['target']
        
        result = target(**input_data)
        
        if 'post_process' in action:
            post_process = action['post_process']
            cache.update(post_process(result))    
        
    return issue_list
run_test(ctf)


In [ ]:
def test_save(testlist):
    f = open("archive/test-"+datetime.datetime.now().strftime("%y-%m-%d-%H-%M")+".txt", "w")
    for item in testlist:
        f.write("%s\n" % item)
    f.close()
test_save(apitestlist)